In [ ]:
#Importo le librerie necessarie per la gestione del dataset
import pandas as pd
from sklearn.model_selection import train_test_split

**Analisi dataset**

In [ ]:
#Monto il drive
from google.colab import drive
drive.mount('/content/drive')
dataset_path = "/content/drive/MyDrive/machine learning"

In [ ]:
#Carico il dataset
dataset = pd.read_csv(dataset_path + "/diabetes.csv")

In [ ]:
#Mostro le prime righe del dataset
dataset.head()

In [ ]:
#Mostro le statistiche descrittive
print("Statistiche descrittive:")
print(dataset.describe())

In [ ]:
#Rimuovo i dati uguali a 0 in colonne in cui un tale risultato sarebbe improbabile
dataset = dataset[(dataset[['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']] != 0).all(axis=1)]

In [ ]:
#Divido il dataset in X (features) e y (target)
X = dataset.drop(columns=['Outcome'])
y = dataset['Outcome']

In [ ]:
#Divido il dataset in training set e test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**Decision Tree**

In [ ]:
#Importo le librerie necessarie
import numpy as np
from scipy import stats
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, roc_curve

In [ ]:
#Calcolo l'entropia
def entropy(y):
    #Conto le occorrenze delle classi
    y_unique, y_unique_count = np.unique(y, return_counts=True)
    #Calcolo le probabilità
    probs = y_unique_count / len(y)
    #Calcolo l'entropia in base 2
    entropy_value = stats.entropy(probs, base=2)
    return entropy_value

#Calcolo l'entropia per la colonna Outcome (le classi)
outcome_entropy = entropy(dataset['Outcome'])
#Stampo il risultato
print(f"L'entropia del dataset per la colonna Outcome è: {outcome_entropy}")

In [ ]:
#Creo il modello Decision Tree
dt_model = DecisionTreeClassifier(class_weight='balanced', criterion='entropy', max_depth=4, min_samples_split=10, min_samples_leaf=5, random_state=42)

In [ ]:
#Addestro il modello
dt_model.fit(X_train, y_train)

In [ ]:
#Valuto le prestazioni del modello sulle caratteristiche del test set
dt_y_pred = dt_model.predict(X_test)

In [ ]:
#Calcolo la probabilità che ogni riga del test set appartenga alla classe 1
dt_y_pred_prob = dt_model.predict_proba(X_test)[:, 1]

In [ ]:
#Calcolo il report di classificazione
print("Report di Classificazione - Decision Tree:")
print(classification_report(y_test, dt_y_pred, target_names=['Non-Diabete', 'Diabete']))

In [ ]:
#Ottengo l'importanza delle features dal modello Decision Tree
#Creo una tabella con due colonne: Feature (caratteristiche) e Importance (importanza delle caratteristiche)
dt_feature_importance = pd.DataFrame({'Feature': X.columns, 'Importance': dt_model.feature_importances_})
#Ordino le features sulla base dell'importanza
dt_feature_importance = dt_feature_importance.sort_values(by='Importance', ascending=False)
#Stampo l'importanza delle features
print("Importanza delle Feature - Decision Tree:")
print(dt_feature_importance)

#Visualizzo l'importanza delle features
#Stabilizzo la dimensione del grafico
plt.figure(figsize=(10, 6))
#Creo un grafico a barre orizzontali
plt.barh(dt_feature_importance['Feature'], dt_feature_importance['Importance'], color='skyblue')
#Inserisco il titolo
plt.title("Decision Tree - Importanza delle Feature")
#Mostro la caratteristica più importante in cima
plt.gca().invert_yaxis()
#Mostro il grafico
plt.show()

In [ ]:
#Calcolo il punteggio ROC-AUC
roc_auc = roc_auc_score(y_test, dt_y_pred_prob)
print(f"ROC-AUC Score: {roc_auc:.2f}")

In [ ]:
#Visualizzo la curva ROC
#Calcolo i dati per la curva ROC
fpr, tpr, thresholds = roc_curve(y_test, dt_y_pred_prob)
#Definisco la dimensione del grafico
plt.figure(figsize=(8, 6))
#Disegno la curva ROC
plt.plot(fpr, tpr, label=f"ROC Curve (AUC = {roc_auc:.2f})")
#Aggiungo una riga di riferimento
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
#Definisco asse x e y
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
#Inserisco il titolo
plt.title("Curva ROC - Decision Tree")
#Mostro la legenda e il grafico
plt.legend()
plt.show()

In [ ]:
#Visualizzo l'albero di decisione
#Imposto la dimensione
plt.figure(figsize=(15, 10))
#Disegno l'albero decisionale specificandone le caratteristiche grafiche
plot_tree(dt_model, filled=True, feature_names=X_train.columns, class_names=['Non Diabete', 'Diabete'], rounded=True, fontsize=12)
#Mostro l'albero
plt.show()

In [ ]:
#Verifico il funzionamento su nuovi dati
new_data = pd.DataFrame([[3, 150, 85, 30, 120, 32.5, 0.7, 45]],
                        columns=['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness',
                                 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age'])

#Predico le probabilità di sviluppo del diabete
probabilities = dt_model.predict_proba(new_data)

#Predico la probabilità di non sviluppare il diabete (Classe 0)
probability_not_diabetes = probabilities[0][0]

#Predico la probabilità di sviluppare il diabete (Classe 1)
probability_diabetes = probabilities[0][1]

#Stampo i risultati
print(f"Probabilità di NON sviluppare diabete: {probability_not_diabetes * 100:.2f}%")
print(f"Probabilità di sviluppare diabete: {probability_diabetes * 100:.2f}%")


**Regressione Logistica**

In [ ]:
#Importo le librerie necessarie
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve

In [ ]:
#Standardizzo i dati
#Definisco l'oggetto scaler
scaler = StandardScaler()
#Standardizzo il train set
X_train_scaled = scaler.fit_transform(X_train)
#Standardizzo il test set
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Stampo i primi 5 record dell'array standardizzato
print("Prime cinque righe di X_train standardizzato:")
print(X_train_scaled[:5])

print("Prime righe di X_test standardizzato:")
print(X_test_scaled[:5])


In [ ]:
#Inizializzo il modello di regressione logistica
rl_model = LogisticRegression(penalty='l2', C = 1.0, class_weight = 'balanced', random_state=42)

In [ ]:
#Addestro il modello
rl_model.fit(X_train_scaled, y_train)

In [ ]:
#Calcolo la probabilità che ogni riga dal test set appartenga alla classe 1
rl_y_pred_prob = rl_model.predict_proba(X_test_scaled)[:, 1]

In [ ]:
#Calcolo le predizioni sul test set
rl_y_pred = rl_model.predict(X_test_scaled)

In [ ]:
#Calcolo il report di classificazione
print("Report di Classificazione - Regressione Logistica:")
print(classification_report(y_test, rl_y_pred, target_names=['Non-Diabete', 'Diabete']))

In [ ]:
#Visualizzo l'importanza delle features
#Creo una tabella con due colonne: una con le features (caratteristiche) e una con i valori assoluti dei coefficienti
rl_feature_importance = pd.DataFrame({'Feature': X.columns, 'Importance': np.abs(rl_model.coef_[0])})
#Ordino la tabella per importanza
rl_feature_importance = rl_feature_importance.sort_values(by='Importance', ascending=False)
#Stampo la tabella
print("Importanza delle Feature - Regressione Logistica:")
print(rl_feature_importance)

#Visualizzo l'importanza delle feature
#Definisco la dimensione del grafico
plt.figure(figsize=(10, 6))
#Creo un grafico a barre orizzontali
plt.barh(rl_feature_importance['Feature'], rl_feature_importance['Importance'], color='lightgreen')
#Definisco il titolo
plt.title("Regressione Logistica - Importanza delle Feature")
#Metto le caretteristiche più importanti in cima
plt.gca().invert_yaxis()
#Mostro il grafico
plt.show()

In [ ]:
#Calcolo il punteggio ROC-AUC
roc_auc = roc_auc_score(y_test, rl_y_pred_prob)
print(f"ROC-AUC Score: {roc_auc:.2f}")

In [ ]:
#Visualizzo la curva ROC
#Calcolo i dati per la curva ROC
fpr, tpr, thresholds = roc_curve(y_test, rl_y_pred_prob)
#Definisco la dimensione del grafico
plt.figure(figsize=(8, 6))
#Disegno la curva ROC
plt.plot(fpr, tpr, label=f"ROC Curve (AUC = {roc_auc:.2f})")
#Aggiungo una riga di riferimento
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
#Definisco asse x e y
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
#Inserisco il titolo
plt.title("Curva ROC - Regressione Logistica")
#Mostro la legenda e il grafico
plt.legend()
plt.show()

In [ ]:
#Verifico il funzionamento su nuovi dati
new_data = pd.DataFrame([[3, 150, 85, 30, 120, 32.5, 0.7, 45]],
                        columns=['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness',
                                 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age'])

#Standardizzo i nuovi dati inseriti
new_data_scaled = scaler.transform(new_data)

#Predico la probabilità di sviluppare o meno il diabete
probabilities = rl_model.predict_proba(new_data_scaled)

#Estraggo le probabilità di non avere o avere il diabete
probability_no_diabetes = probabilities[0][0]
probability_diabetes = probabilities[0][1]

#Stampo i risultati
print(f"Probabilità di NON avere il diabete: {probability_no_diabetes * 100:.2f}%")
print(f"Probabilità di avere il diabete: {probability_diabetes * 100:.2f}%")

**Support Vector Machine Lineare**

In [ ]:
#Importo le librerie necessarie
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score

In [ ]:
#Inizializzo il modello svm
svm_model = SVC(kernel='linear', probability=True, class_weight='balanced', random_state=42)

In [ ]:
#Addestro il modello svm
svm_model.fit(X_train_scaled, y_train)

In [ ]:
#Eseguo le previsioni sul test set
svm_y_pred = svm_model.predict(X_test_scaled)

In [ ]:
#Calcolo la probabilità che ogni riga dal test set appartenga alla classe 1
svm_y_pred_prob = svm_model.predict_proba(X_test_scaled)[:, 1]

In [ ]:
#Calcolo il report di classificazione
print("Report di Classificazione - Support Vector Machine Lineare:")
print(classification_report(y_test, svm_y_pred, target_names=['Non-Diabete', 'Diabete']))

In [ ]:
#Visualizzo l'importanza delle feature
#Creo una tabella con due colonne: Feature contenente le caratteristiche e Importance contenente i coefficienti del modello
svm_feature_importance = pd.DataFrame({'Feature': X.columns, 'Importance': np.abs(svm_model.coef_[0])})
#Ordino le features per importanza
svm_feature_importance = svm_feature_importance.sort_values(by='Importance', ascending=False)
#Stampo la tabella
print("Importanza delle Features - Support Vector Machine Lineare:")
print(svm_feature_importance)

#Visualizzo l'importanza delle feature
#Definisco la dimensione del grafico
plt.figure(figsize=(10, 6))
#Creo un grafico a barre orizzontali
plt.barh(svm_feature_importance['Feature'], svm_feature_importance['Importance'], color='blue')
#Definisco un titolo
plt.title("Support Vector Machine Lineare - Importanza delle Features")
#Metto in cima le caratteristiche più importanti
plt.gca().invert_yaxis()
#Mostro il grafico
plt.show()

In [ ]:
#Calcolo il punteggio ROC-AUC
roc_auc = roc_auc_score(y_test, svm_y_pred_prob)
print(f"ROC-AUC Score: {roc_auc:.2f}")

In [ ]:
#Visualizzo la curva ROC
#Calcolo i dati per la curva ROC
fpr, tpr, thresholds = roc_curve(y_test, svm_y_pred_prob)
#Definisco la dimensione del grafico
plt.figure(figsize=(8, 6))
#Disegno la curva ROC
plt.plot(fpr, tpr, label=f"ROC Curve (AUC = {roc_auc:.2f})")
#Aggiungo una riga di riferimento
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
#Definisco asse x e y
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
#Inserisco il titolo
plt.title("Curva ROC - SVM Lineare")
#Mostro la legenda e il grafico
plt.legend()
plt.show()

In [ ]:
#Verifico il funzionamento su nuovi dati
new_data = pd.DataFrame([[3, 150, 85, 30, 120, 32.5, 0.7, 45]],
                        columns=['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness',
                                 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age'])

#Standardizzo i nuovi dati inseriti
new_data_scaled = scaler.transform(new_data)

#Predico la probabilità di sviluppare o meno il diabete
probabilities = svm_model.predict_proba(new_data_scaled)

#Estraggo le probabilità di non avere o avere il diabete
probability_no_diabetes = probabilities[0][0]
probability_diabetes = probabilities[0][1]

#Stampo i risultati
print(f"Probabilità di NON avere il diabete: {probability_no_diabetes * 100:.2f}%")
print(f"Probabilità di avere il diabete: {probability_diabetes * 100:.2f}%")

**SVM non lineare**

In [ ]:
import shap

In [ ]:
#Inizializzo il modello svm non lineare
svm_rbf_model = SVC(kernel='rbf', probability=True, class_weight='balanced', random_state=42)

In [ ]:
#Addestro il modello
svm_rbf_model.fit(X_train_scaled, y_train)

In [ ]:
#Eseguo le previsioni sul test set
svm_rbf_y_pred = svm_rbf_model.predict(X_test_scaled)

In [ ]:
#Calcolo la probabilità che ogni riga dal test set appartenga alla classe 1
svm_rbf_y_pred_prob = svm_rbf_model.predict_proba(X_test_scaled)[:, 1]

In [ ]:
#Calcolo il report di classificazione
print("Report di Classificazione - Support Vector Machine Non Lineare:")
print(classification_report(y_test, svm_rbf_y_pred, target_names=['Non-Diabete', 'Diabete']))

In [ ]:
#Calcolo l'importanza delle features
#Uso i dati di set per capire come cambieranno le predizioni se cambio le features
explainer = shap.KernelExplainer(svm_rbf_model.predict, X_test_scaled)
#Calcolo quanto ogni features contribuisce alle predizioni
shap_values = explainer.shap_values(X_test_scaled)
#Creo un grafico che mostri l'importanza delle features
shap.summary_plot(shap_values, X_test_scaled, feature_names=X.columns)

In [ ]:
#Calcolo il punteggio ROC-AUC
roc_auc = roc_auc_score(y_test, svm_rbf_y_pred_prob)
print(f"ROC-AUC Score: {roc_auc:.2f}")

In [ ]:
#Visualizzo la curva ROC
#Calcolo i dati per la curva ROC
fpr, tpr, thresholds = roc_curve(y_test, svm_rbf_y_pred_prob)
#Definisco la dimensione del grafico
plt.figure(figsize=(8, 6))
#Disegno la curva ROC
plt.plot(fpr, tpr, label=f"ROC Curve (AUC = {roc_auc:.2f})")
#Aggiungo una riga di riferimento
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
#Definisco asse x e y
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
#Inserisco il titolo
plt.title("Curva ROC - SVM Non Lineare")
#Mostro la legenda e il grafico
plt.legend()
plt.show()

In [ ]:
#Verifico il funzionamento su nuovi dati
new_data = pd.DataFrame([[3, 150, 85, 30, 120, 32.5, 0.7, 45]],
                        columns=['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness',
                                 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age'])

#Standardizzo i nuovi dati inseriti
new_data_scaled = scaler.transform(new_data)

#Predico la probabilità di sviluppare o meno il diabete
probabilities = svm_rbf_model.predict_proba(new_data_scaled)

#Estraggo le probabilità di non avere o avere il diabete
probability_no_diabetes = probabilities[0][0]
probability_diabetes = probabilities[0][1]

#Stampo i risultati
print(f"Probabilità di NON avere il diabete: {probability_no_diabetes * 100:.2f}%")
print(f"Probabilità di avere il diabete: {probability_diabetes * 100:.2f}%")